In [55]:
!pip install transformers

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

I have taken the dataset from here(https://github.com/clairett/pytorch-sentiment-classification/tree/master/data/SST2)

In [57]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [58]:
df.shape

(6920, 2)

 we'll only use 2,000 sentences from the dataset

In [59]:
batch_1 = df[:2000]

In [60]:
batch_1

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
1995,too bland and fustily tasteful to be truly pru...,0
1996,it does n't work as either,0
1997,this one aims for the toilet and scores a dire...,0
1998,in the name of an allegedly inspiring and easi...,0


In [61]:
batch_1[0]

0       a stirring , funny and finally transporting re...
1       apparently reassembled from the cutting room f...
2       they presume their audience wo n't sit still f...
3       this is a visually stunning rumination on love...
4       jonathan parker 's bartleby should have been t...
                              ...                        
1995    too bland and fustily tasteful to be truly pru...
1996                           it does n't work as either
1997    this one aims for the toilet and scores a dire...
1998    in the name of an allegedly inspiring and easi...
1999    the movie is undone by a filmmaking methodolog...
Name: 0, Length: 2000, dtype: object

We can ask pandas how many sentences are labeled as "positive" (value 1) and how many are labeled "negative" (having the value 0)

In [62]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Loading the Pre-trained BERT model


In [ ]:
## For BERT
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)



## Model #1: Preparing the Dataset

### Tokenization


In [64]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [65]:
tokenized# every sentence into the list of ids

0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
1995    [101, 2205, 20857, 1998, 11865, 16643, 2135, 5...
1996    [101, 2009, 2515, 1050, 1005, 1056, 2147, 2004...
1997    [101, 2023, 2028, 8704, 2005, 1996, 11848, 199...
1998    [101, 1999, 1996, 2171, 1997, 2019, 9382, 1898...
1999    [101, 1996, 3185, 2003, 25757, 2011, 1037, 244...
Name: 0, Length: 2000, dtype: object

In [66]:
tokenized.values

array([list([101, 1037, 18385, 1010, 6057, 1998, 2633, 18276, 2128, 16603, 1997, 5053, 1998, 1996, 6841, 1998, 5687, 5469, 3152, 102]),
       list([101, 4593, 2128, 27241, 23931, 2013, 1996, 6276, 2282, 2723, 1997, 2151, 2445, 12217, 7815, 102]),
       list([101, 2027, 3653, 23545, 2037, 4378, 24185, 1050, 1005, 1056, 4133, 2145, 2005, 1037, 11507, 10800, 1010, 2174, 14036, 2135, 3591, 1010, 2061, 2027, 19817, 4140, 2041, 1996, 7511, 2671, 4349, 3787, 1997, 11829, 7168, 9219, 1998, 28971, 2308, 1999, 8301, 8737, 2100, 4253, 102]),
       ...,
       list([101, 2023, 2028, 8704, 2005, 1996, 11848, 1998, 7644, 1037, 3622, 2718, 102]),
       list([101, 1999, 1996, 2171, 1997, 2019, 9382, 18988, 1998, 4089, 3006, 3085, 17312, 1010, 1996, 3750, 1005, 1055, 2252, 4332, 1037, 6397, 3239, 2000, 1996, 2200, 2381, 2009, 9811, 2015, 2000, 6570, 102]),
       list([101, 1996, 3185, 2003, 25757, 2011, 1037, 24466, 16134, 2008, 1005, 1055, 2074, 6388, 2438, 2000, 7344, 3686, 1996, 7731, 4378, 209

# padding

In [67]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [68]:
padded

array([[  101,  1037, 18385, ...,     0,     0,     0],
       [  101,  4593,  2128, ...,     0,     0,     0],
       [  101,  2027,  3653, ...,     0,     0,     0],
       ...,
       [  101,  2023,  2028, ...,     0,     0,     0],
       [  101,  1999,  1996, ...,     0,     0,     0],
       [  101,  1996,  3185, ...,     0,     0,     0]])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [69]:
np.array(padded).shape

(2000, 59)

### Masking


In [70]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [71]:
# We now create an input tensor out of the padded token matrix, 
# and send that to BERT
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():#temporarily set all the requires_grad flag to false
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [72]:
last_hidden_states[0].shape

torch.Size([2000, 59, 768])

In [73]:
last_hidden_states[0][:,0,:].shape

torch.Size([2000, 768])

In [74]:
# Slice the output for the first position for all the sequences, take all hidden unit outputs
last_hidden_states[0][:,0,:].numpy()

array([[-0.5566494 , -0.33129224, -0.22280575, ..., -0.22786134,
         0.63191915,  0.2430667 ],
       [-0.28789234, -0.14285505, -0.06857931, ..., -0.316906  ,
         0.18455267,  0.31989855],
       [-0.18645243,  0.30229482, -0.18511146, ..., -0.3349294 ,
         0.9848731 ,  0.5297742 ],
       ...,
       [-0.7284262 , -0.09083486, -0.12268987, ...,  0.11295995,
         0.38278908,  0.77147824],
       [-0.08991001,  0.41575524, -0.1809646 , ..., -0.23456128,
         0.39538246,  0.5656696 ],
       [ 0.23998241,  0.2008787 ,  0.11803265, ..., -0.18060395,
         0.36744034,  0.14869873]], dtype=float32)

In [75]:
features = last_hidden_states[0][:,0,:].numpy()

* And now features is a 2d numpy array containing the sentence embeddings of all the sentences in our dataset.

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [76]:
labels = batch_1[1]

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set 

In [77]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

We now train the LogisticRegression model. 

In [78]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


## Evaluating Model 
accuracy against the testing dataset:

In [79]:
lr_clf.score(test_features, test_labels)

0.832

In [80]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

 Let's first look at a dummy classifier:

#### DummyClassifier is a classifier that makes predictions using simple rules.

#### This classifier is useful as a simple baseline to compare with other (real) classifiers.

In [81]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels,cv = 5)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.509 (+/- 0.06)
